In [39]:
import helpers
import numpy as np
import re 

#helpers.npNicePrint(arr)

data = """Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=8400, Y=5400

Button A: X+26, Y+66
Button B: X+67, Y+21
Prize: X=12748, Y=12176

Button A: X+17, Y+86
Button B: X+84, Y+37
Prize: X=7870, Y=6450

Button A: X+69, Y+23
Button B: X+27, Y+71
Prize: X=18641, Y=10279"""
#data = """125 17"""
with open("input_13.txt", 'r') as f:
    data = f.read()


def safe_get(grid, index):
    try:
        return grid[tuple(index)]
    except:
        return ""
    
#changeDict = {"forward": np.array([1,0]), "down": np.array([0,1]), "up": np.array([0,-1])}
#data = [changeDict[x.split(" ")[0]] * int(x.split(" ")[1]) for x in data.split("\n") if x]
#data = [
#    [int(match.group(1)), list(map(int, match.group(2).split()))]
#    for line in data.split("\n")
#    if (match := re.match(r'^(\d+):\s*(\d+(?:\s+\d+)*)', line))
#]
#np.prod(sum(data))    
#data = [list(map(int, x.split(" "))) for x in data.split('\n') if not x == ""]
#grid = {(x,y): str(val) for y, r in enumerate(data.split('\n')) for x, val in enumerate(r)}
#grid
#data = [int(x) for x in data.split(" ")]

# Regex to capture the data
pattern = r"Button A: X\+(\d+), Y\+(\d+)\s+Button B: X\+(\d+), Y\+(\d+)\s+Prize: X=(\d+), Y=(\d+)"
matches = re.findall(pattern, data)

# Parse the data into numpy-compatible format
data = [
    np.array([(int(a_x), int(a_y)), (int(b_x), int(b_y)), (int(p_x), int(p_y))])
    for a_x, a_y, b_x, b_y, p_x, p_y in matches
]

# Output the parsed data
#print(data)



In [ ]:
#I'm keeping all the cells here to show how much of a massive PITA it was to find a solver that would work in the end...
#Trying with cycles first, which didn't work, then switched to solving an ILP program, which turned out to be more annoying
#than expected...

In [104]:
A, B, target = data[0]
target = target.copy()
tokens = 0
last_tokens = 0
A_cycle = [0,0]
B_cycle = [0,0]
while True:
    target -= B
    tokens += 1
    if target[0] % A[0] == 0 and target[1] % A[1] == 0:
        print(tokens-last_tokens)
        B_cycle[0] = tokens-last_tokens
        last_tokens = tokens
        break

while True:
    target -= B
    tokens += 1
    if target[0] % A[0] == 0 and target[1] % A[1] == 0:
        print(tokens-last_tokens)
        B_cycle[1] = tokens-last_tokens
        last_tokens = tokens
        break
        
#everytime we press B B_cycle[1] times after pressing B B_cycle[0] times once, target is a multiple of A i.e. we can press A x times to get target
        
last_tokens = 0
tokens = 0
            
while True:
    target -= A
    tokens += 1
    if target[0] % B[0] == 0 and target[1] % B[1] == 0:
        print(tokens-last_tokens)
        A_cycle[0] = tokens-last_tokens
        last_tokens = tokens
        break
        

while True:
    target -= A
    tokens += 1
    if target[0] % B[0] == 0 and target[1] % B[1] == 0:
        print(tokens-last_tokens)
        A_cycle[1] = tokens-last_tokens
        last_tokens = tokens
        break
        
#everytime we press A A_cycle[1] times after pressing A A_cycle[0] times once, target is a multiple of A i.e. we can press A x times to get target
        
B_cycle, A_cycle

46
1598
386
737


([46, 1598], [386, 737])

In [97]:
tokens = A_cycle[0] * 3 + B_cycle[0]
A, B, target = data[0]
target = target.copy()
print(target)
target -= (A_cycle[0] * A) + (B_cycle[0] * B)
target

[10000000008400 10000000005400]


array([9999999985214, 9999999986504], dtype=int64)

In [105]:
9999999985214 / 1598

6257822268.594493

In [99]:
sum_AB = (469 * A) + (286 * B)
target[0] % sum_AB[0], target[1] % sum_AB[1] 

(33182, 27192)

In [76]:
sum_AB

array([104434, 132124], dtype=int64)

In [98]:
target[0] % A[0], target[1] % A[1], target[0] % B[0], target[1] % B[1]

(10, 30, 8, 57)

In [20]:
A, B, target = data[0]
tokens = 0
while not target[0] % A[0] == 0 and not target[1] % A[1] == 0:
    target -= B
    tokens += 1
    if target[0] % A[0] == 0 and target[1] % A[1] == 0:
        print (tokens)
tokens

[ 9778 15506]
[ 9722 15483]
[ 9666 15460]
[ 9610 15437]
[ 9554 15414]
[ 9498 15391]
[ 9442 15368]
[ 9386 15345]


8

In [62]:
from math import lcm
lcm(1598, 737)

1177726

In [102]:
from math import gcd
from itertools import product

# Inputs
A = [94, 34]
B = [22, 67]
target = [10000000008400, 10000000005400]
cost_A = 3
cost_B = 1

def extended_gcd(a, b):
    if b == 0:
        return a, 1, 0
    gcd, x1, y1 = extended_gcd(b, a % b)
    x = y1
    y = x1 - (a // b) * y1
    return gcd, x, y

def find_min_cost(A, B, target, cost_A, cost_B):
    # Step 1: Reduce target modulo GCD of A and B
    gcd_x = gcd(A[0], B[0])
    gcd_y = gcd(A[1], B[1])
    
    if target[0] % gcd_x != 0 or target[1] % gcd_y != 0:
        return "Target not reachable"
    
    # Scale down A, B, and target
    A = [A[0] // gcd_x, A[1] // gcd_y]
    B = [B[0] // gcd_x, B[1] // gcd_y]
    target = [target[0] // gcd_x, target[1] // gcd_y]

    # Step 2: Use extended Euclidean to find integer solutions
    gcd_a, x_a, y_a = extended_gcd(A[0], B[0])
    gcd_b, x_b, y_b = extended_gcd(A[1], B[1])
    
    # Check all combinations
    min_cost = float('inf')
    for x_offset, y_offset in product(range(100), range(100)):
        # Solve for integer x and y
        x = x_a * target[0] + x_offset * B[0]
        y = y_b * target[1] + y_offset * A[1]
        
        # Compute cost
        cost = cost_A * x + cost_B * y
        if cost < min_cost:
            min_cost = cost

    return min_cost

# Compute the minimum cost
result = find_min_cost(A, B, target, cost_A, cost_B)
print("Minimum cost:", result)


Minimum cost: 50000000045000


In [103]:
A, B, target = data[1]
A, B, target

(array([26, 66], dtype=int64),
 array([67, 21], dtype=int64),
 array([10000000012748, 10000000012176], dtype=int64))

In [120]:
A, B, target = data[1]
target = target.copy()
tokens = 0
last_tokens = 0
A_cycle = [0,0]
B_cycle = [0,0]
while True:
    target -= B
    tokens += 1
    if target[0] % A[0] == 0 and target[1] % A[1] == 0:
        if (target[0] / A[0]) == (target[1] / A[1]):
            print(tokens-last_tokens)
            B_cycle[0] = tokens-last_tokens
            last_tokens = tokens
            break

while True:
    target -= B
    tokens += 1
    if target[0] % A[0] == 0 and target[1] % A[1] == 0:
        if (target[0] / A[0]) == (target[1] / A[1]):
            print(tokens-last_tokens)
            B_cycle[1] = tokens-last_tokens
            last_tokens = tokens
            break
        


KeyboardInterrupt: 

In [115]:
target, A

(array([9999994327438, 9999997853518], dtype=int64),
 array([94, 34], dtype=int64))

In [117]:
9999994327438 % 94, 9999997853518 % 34

(0, 0)

In [118]:
(9999994327438 / 94) == (9999997853518 / 34)

(106382918377.0, 294117583927.0)

In [112]:
while True:
    target -= A
    print(target[0] % A[0] == 0,target[1] % A[1] == 0)

True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


...

True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


KeyboardInterrupt: 

In [ ]:
#find x, y s.t. the following equation is true
x * A[0] + y * B[0] = target[0]
x * A[1] + y * B[1] = target[1]
total = 3*x + y

In [121]:
# Define the matrices
A, B, target = data[1]
target = target.copy()

# Create the matrix M and the vector b
M = np.array([[A[0], B[0]], [A[1], B[1]]])
b = np.array([target[0], target[1]])

# Solve the system M * [x, y] = b
x, y = np.linalg.solve(M, b)

print(f"x = {x}, y = {y}")

x = 118679050709.0, y = 103199174542.00002


In [123]:
az = x * A + y * B

In [126]:
az[0], az[1]

(10000000012748.0, 10000000012176.0)

In [125]:
target

array([10000000012748, 10000000012176], dtype=int64)

In [183]:
from scipy.optimize import linprog
ans = 0
for A, B, target in data:
    # Define the coefficients of the objective function (3x + y)
    c = [3, 1]  # Minimize 3x + y

    # Coefficients of the equality constraints
    # A*x + B*y = target
    A_eq = [
        [A[0], B[0]],  # A[0]*x + B[0]*y = target[0]
        [A[1], B[1]]   # A[1]*x + B[1]*y = target[1]
    ]

    # Right-hand side of the equality constraints
    b_eq = [target[0], target[1]]

    # Use linprog to minimize the objective function
    result = linprog(c, A_eq=A_eq, b_eq=b_eq, method='highs')

    # Output the result
    if result.success:
        x, y = result.x
        if (round(x) * A[0] + int(y) * B[0] == target[0]) and (round(x) * A[1] + int(y) * B[1] == target[1]):
            print(f"Optimal solution: x = {x}, y = {y}")
            print(f"Minimum total: {result.fun}")
            ans += int(result.fun)
        else:
            print(f"No solution found. {result.fun}")
    else:
        print(f"2 2 2 2 No solution found. {result.fun}")
ans

Optimal solution: x = 258014073998.0, y = 109460516068.0
Minimum total: 883502738062.0
No solution found. 794415021932.8643
No solution found. 401087695541.2753
Optimal solution: x = 188679245596.0, y = 165094339790.0
Minimum total: 731132076578.0
Optimal solution: x = 100612423577.99998, y = 146544182115.0
Minimum total: 448381452848.99994
Optimal solution: x = 154867257004.0, y = 188053097140.0
Minimum total: 652654868152.0
Optimal solution: x = 114035087944.0, y = 206140350882.0
Minimum total: 548245614714.0
Optimal solution: x = 200163398794.0, y = 106209150566.0
Minimum total: 706699346948.0
Optimal solution: x = 231607629895.00003, y = 129427792897.0
Minimum total: 824250682582.0001
Optimal solution: x = 184300341341.99997, y = 116040955638.0
Minimum total: 668941979663.9999
No solution found. 411815438246.0
Optimal solution: x = 105803142034.0, y = 102596986387.0
Minimum total: 420006412489.0
No solution found. 439144003882.65186
Optimal solution: x = 128369704995.0, y = 1155327

95311181265585

In [ ]:
#95311181265585 too low

In [182]:
round(470588236371.5)

470588236372

In [36]:
import pulp


for A, B, target in data:
    # Define the problem as a minimization problem
    problem = pulp.LpProblem("Minimize_total", pulp.LpMinimize)

    # Define the decision variables x and y as integers
    x = pulp.LpVariable('x', lowBound=0, cat='Integer')  # x >= 0
    y = pulp.LpVariable('y', lowBound=0, cat='Integer')  # y >= 0

    # Objective function: minimize 3x + y
    problem += 3 * x + y, "Total"

    # Constraints: A[0] * x + B[0] * y = target[0]
    problem += int(A[0]) * x + int(B[0]) * y == int(target[0]), "Constraint 1"
    problem += int(A[1]) * x + int(B[1]) * y == int(target[1]), "Constraint 2"

    # Solve the problem
    problem.solve(pulp.SCIP_CMD())

    # Output the results
    if pulp.LpStatus[problem.status] == 'Optimal':
        print(f"Optimal solution: x = {x.varValue}, y = {y.varValue}")
        print(f"Minimum total: {pulp.value(problem.objective)}")
    else:
        print("No optimal solution found.")


PulpSolverError: PuLP: cannot execute scip.exe

In [26]:
!pip install highspy

In [27]:
import highspy as hs

# Assuming 'data' contains the input data as (A, B, target)
for A, B, target in data:
    # Define the problem as a minimization problem
    problem = hs.LpProblem("Minimize_total", hs.LpMinimize)

    # Define the decision variables x and y as integers
    x = hs.LpVariable('x', lowBound=0, cat='Integer')  # x >= 0
    y = hs.LpVariable('y', lowBound=0, cat='Integer')  # y >= 0

    # Objective function: minimize 3x + y
    problem += 3 * x + y, "Total"

    # Constraints: A[0] * x + B[0] * y = target[0]
    problem += int(A[0]) * x + int(B[0]) * y == int(target[0]), "Constraint 1"
    problem += int(A[1]) * x + int(B[1]) * y == int(target[1]), "Constraint 2"

    # Solve the problem
    problem.solve(hs.COIN_CMD())

    # Output the results
    if problem.status == hs.LpStatus.Optimal:
        print(f"Optimal solution: x = {x.varValue}, y = {y.varValue}")
        print(f"Minimum total: {problem.objective.value()}")
    else:
        print("No optimal solution found.")

TypeError: 'numpy._DTypeMeta' object is not subscriptable

In [30]:
import highspy as hs

TypeError: 'numpy._DTypeMeta' object is not subscriptable

In [31]:
pulp.listSolvers(onlyAvailable=True)

['PULP_CBC_CMD']

In [33]:
import pulp

for A, B, target in data:
    # Create the ILP problem (minimization or maximization)
    problem = pulp.LpProblem("ILP_Problem", pulp.LpMinimize)

    # Define decision variables
    x = pulp.LpVariable('x', lowBound=0, cat='Integer')
    y = pulp.LpVariable('y', lowBound=0, cat='Integer')

    # Define the constraints
    problem += A[0] * x + B[0] * y == target[0], "Constraint1"
    problem += A[1] * x + B[1] * y == target[1], "Constraint2"

    # Define the objective function
    problem += 3 * x + y, "Objective"

    # Solve the problem
    problem.solve()

    # Output the results
    print(f"Status: {pulp.LpStatus[problem.status]}")
    print(f"x = {x.varValue}")
    print(f"y = {y.varValue}")

Status: Infeasible
x = 81081081000.0
y = 108108110000.0
Status: Infeasible
x = 118679050000.0
y = 103199170000.0
Status: Infeasible
x = 71266111000.0
y = 104624720000.0
Status: Infeasible
x = 102851800000.0
y = 107526880000.0


In [35]:
# Install the necessary dependencies
#!apt-get install -y build-essential python3-dev gfortran

# Install SCIP and pySCIPOpt
!pip install pyscipopt

In [40]:
import pyscipopt
ans = 0
error = 10000000000000
answers = []
for A, B, target in data:

  # Create a simple SCIP model to test the installation
  model = pyscipopt.Model("ILP_Model") 

  # Add some variables
  x = model.addVar("x", lb=0, vtype="INTEGER")
  y = model.addVar("y", lb=0, vtype="INTEGER")

  # Add a constraint
  model.addCons(A[0]*x + B[0]*y == target[0] + error)
  model.addCons(A[1]*x + B[1]*y == target[1] + error)

  # Set the objective function
  model.setObjective(3*x + y, sense="minimize")

  # Optimize the model
  model.optimize()

  # Print the results
  if model.getStatus() == "optimal":
      print("Optimal solution found!")
      print("x = ", model.getVal(x))
      print("y = ", model.getVal(y))
      #print(model.getVal(x) * A[0] + model.getVal(y) * B[0],target[0])
      #print(model.getVal(x) * A[1] + model.getVal(y) * B[1],target[1])
      if model.getVal(x) * A[0] + model.getVal(y) * B[0] == target[0]+ error and model.getVal(x) * A[1] + model.getVal(y) * B[1] == target[1]+ error:
        print("Solution is correct!")
        ans += (model.getObjVal())
        answers.append([round(model.getVal(x)), round(model.getVal(y))])
      else:
        print("Solution is incorrect!")
      
  else:
      print("No optimal solution found.")
ans

Optimal solution found!
x =  258014073998.0
y =  109460516068.0
Solution is correct!
Optimal solution found!
x =  250361097806.0
y =  43331728504.0
Solution is incorrect!
Optimal solution found!
x =  115567641066.0
y =  54384772340.0
Solution is incorrect!
Optimal solution found!
x =  188679245596.0
y =  165094339790.0
Solution is correct!
Optimal solution found!
x =  100612423578.0
y =  146544182115.0
Solution is correct!
Optimal solution found!
x =  154867257004.0
y =  188053097140.0
Solution is correct!
Optimal solution found!
x =  114035087944.0
y =  206140350882.0
Solution is correct!
Optimal solution found!
x =  200163398794.0
y =  106209150566.0
Solution is correct!
Optimal solution found!
x =  231607629895.0
y =  129427792897.0
Solution is correct!
Optimal solution found!
x =  184300341342.0
y =  116040955638.0
Solution is correct!
Optimal solution found!
x =  103492885053.0
y =  101336783087.0
Solution is correct!
Optimal solution found!
x =  105803142034.0
y =  102596986387.0

Optimal solution found!
x =  113141862489.0
y =  108790252415.0
Solution is incorrect!
Optimal solution found!
x =  149253731600.0
y =  126865671879.0
Solution is correct!
Optimal solution found!
x =  103420843463.0
y =  143198090657.0
Solution is correct!
Optimal solution found!
x =  97270160024.0
y =  67461562651.0
Solution is incorrect!
Optimal solution found!
x =  208432023239.0
y =  127901468470.0
Solution is correct!
Optimal solution found!
x =  117753623228.0
y =  181159420347.0
Solution is correct!
Optimal solution found!
x =  161637931065.0
y =  204741379341.0
Solution is incorrect!
Optimal solution found!
x =  156862745050.0
y =  137254902210.0
Solution is correct!
Optimal solution found!
x =  172910663363.0
y =  144092219013.0
Solution is correct!
Optimal solution found!
x =  144792153483.0
y =  112097150937.0
Solution is correct!
No optimal solution found.
Optimal solution found!
x =  119326158430.0
y =  109967243891.0
Solution is correct!
Optimal solution found!
x =  17699

Optimal solution found!
x =  236641221432.0
y =  26717557287.0
Solution is incorrect!
Optimal solution found!
x =  106180234325.0
y =  103457664236.0
Solution is correct!
Optimal solution found!
x =  108583247512.0
y =  134436401187.0
Solution is correct!
Optimal solution found!
x =  123595505620.0
y =  108614232317.0
Solution is correct!
Optimal solution found!
x =  103626943158.0
y =  129533679008.0
Solution is correct!
Optimal solution found!
x =  173661360242.0
y =  101302460481.0
Solution is correct!
Optimal solution found!
x =  131926121019.0
y =  100263852553.0
Solution is correct!
Optimal solution found!
x =  133295519172.0
y =  102098695360.0
Solution is correct!
No optimal solution found.
Optimal solution found!
x =  22145669391.0
y =  98425196880.0
Solution is incorrect!
Optimal solution found!
x =  102803738325.0
y =  112149532807.0
Solution is correct!
Optimal solution found!
x =  39525691791.0
y =  135516657326.0
Solution is incorrect!
Optimal solution found!
x =  1214833

106228669504887.0

In [43]:
#this was a massive PITA to find the correct ILP that would solve it and get all the solutions, most seem to fail in the task
#There also wasn't an easy way to debug since Erik didn't include example correct answers for pt. 2

#not all correct solutions were actually correct, but adding a force check for correctness gives the correct answer in the end
#the actual solution is quite simple, just solving an integer linear program 
#The ILP is as follows:

#find x, y s.t. the following equation is true
#constraint: x * A[0] + y * B[0] = target[0]
#constraint: x * A[1] + y * B[1] = target[1]
#minimize: 3*x + y